In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

/Users/msy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/msy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
image_data = pd.read_pickle('pic_32_32.df')

In [20]:
X=np.stack(image_data.data.values)
Y=np.eye(10)[image_data.label.values]

In [21]:
x_train=X[:9000]
y_train=Y[:9000]
x_test=X[9000:]
y_test=Y[9000:]

# 定义参数

In [24]:
n_classes = 10
batch_size = 128
# batch_turn = 70
padding = 'SAME'
kernel_size_3 = [3,3]
kernel_size_1 = [1,1]
padding_size = [2,2]
keep_prob = 0.5
lr = 1e-4
epoch = 200

# 占坑

In [33]:
x = tf.placeholder(tf.float32, [None, 32,32,3])
y = tf.placeholder(tf.float32, [None, 10])

# stage1

In [35]:
conv_1 = tf.layers.conv2d(
    inputs = x,
    filters = 64,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_2 = tf.layers.conv2d(
    inputs = conv_1,
    filters = 64,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

pool_1 = tf.layers.max_pooling2d(
    inputs = conv_2, 
    pool_size = padding_size, 
    strides = 2,
    padding = padding
)

## stage2

In [36]:
conv_3 = tf.layers.conv2d(
    inputs = pool_1,
    filters = 128,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_4 = tf.layers.conv2d(
    inputs = conv_3,
    filters = 128,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

pool_2 = tf.layers.max_pooling2d(
    inputs = conv_4, 
    pool_size = padding_size, 
    strides = 2,
    padding = padding
)

## stage3

In [37]:
conv_5 = tf.layers.conv2d(
    inputs = pool_2,
    filters = 256,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_6 = tf.layers.conv2d(
    inputs = conv_5,
    filters = 256,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_7 = tf.layers.conv2d(
    inputs = conv_6,
    filters = 256,
    kernel_size = kernel_size_1,
    padding = padding,
    activation = tf.nn.relu    
)

pool_3 = tf.layers.max_pooling2d(
    inputs = conv_7, 
    pool_size = padding_size, 
    strides = 2,
    padding = padding
)

## stage4

In [38]:
conv_8 = tf.layers.conv2d(
    inputs = pool_3,
    filters = 512,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_9 = tf.layers.conv2d(
    inputs = conv_8,
    filters = 512,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_10 = tf.layers.conv2d(
    inputs = conv_9,
    filters = 512,
    kernel_size = kernel_size_1,
    padding = padding,
    activation = tf.nn.relu    
)

pool_4 = tf.layers.max_pooling2d(
    inputs = conv_10, 
    pool_size = padding_size, 
    strides = 2,
    padding = padding
)

## stage5

In [39]:
conv_11 = tf.layers.conv2d(
    inputs = pool_4,
    filters = 512,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_12 = tf.layers.conv2d(
    inputs = conv_11,
    filters = 512,
    kernel_size = kernel_size_3,
    padding = padding,
    activation = tf.nn.relu    
)

conv_13 = tf.layers.conv2d(
    inputs = conv_12,
    filters = 512,
    kernel_size = kernel_size_1,
    padding = padding,
    activation = tf.nn.relu    
)

pool_5 = tf.layers.max_pooling2d(
    inputs = conv_13, 
    pool_size = padding_size, 
    strides = 2,
    padding = padding
)

In [40]:
fc1 = tf.layers.flatten(pool_5)#拍扁：降成一维
fc2 = tf.layers.dense(fc1, 4096, activation=tf.nn.relu)#全连接映射
fc2 = tf.layers.dropout(fc2, rate=keep_prob)
fc3 = tf.layers.dense(fc2, 4096, activation=tf.nn.relu)
fc3 = tf.layers.dropout(fc3, rate=keep_prob)
out = tf.layers.dense(fc3, n_classes)#全连接映射

# 损失函数

In [41]:
cross_loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=out, labels=y)
loss = tf.reduce_mean(cross_loss)
train_step = tf.train.AdamOptimizer(lr).minimize(loss)
correct_prediction = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [31]:
num=int(np.ceil(x_train.shape[0]/batch_size))

# 训练

In [45]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for ep in range(epoch):
        for i in range(num):#8960
            x_batch = x_train[i*batch_size : (i+1)*batch_size]
            y_batch = y_train[i*batch_size : (i+1)*batch_size]
            _,l,ac = sess.run([train_step, loss, accuracy], feed_dict={x:x_batch,y:y_batch})
            if i!=0 and i%10==0:
                print('batch {0} , loss {1} , accuracy {2}'.format(i,l,ac))
        print('----------- the epoch {0} , loss {1} , accuracy {2} -----------'.format(ep,l,ac))
        if ep%10 == 0:
            test_ac = sess.run([accuracy],feed_dict={x:x_test,y:y_test})
            print('*'*10 + 'test accuracy ',test_ac,'*'*10)

(128, 1, 1, 512)
(128, 1, 1, 512)


KeyboardInterrupt: 